<a href="https://colab.research.google.com/github/lizjwh/groupIAI_5take2/blob/main/5%20Completely%20New%20Model/Lizzys_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torchvision
!pip install kaggle
!pip install tqdm
!pip install colorama
!pip install split-folders
! pip install tensorboard

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
torch.manual_seed(0)
from torch import nn, optim, tensor
from torchvision import transforms, models ,datasets
from torch.utils.data import ConcatDataset
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob
from mpl_toolkits.axes_grid1 import ImageGrid
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
import os
import splitfolders

In [3]:
# Check whether we have a GPU.  Use it if we do.
device = torch.device('cuda')

In [4]:
! export KAGGLE_USERNAME="emmet454" && export KAGGLE_KEY="ee00fbc0728a71f5c5f712029e3ef004" && kaggle datasets download --force --unzip emilyburt/intro-to-ai

100% 4.08G/4.08G [02:39<00:00, 39.1MB/s]
100% 4.08G/4.08G [02:39<00:00, 27.4MB/s]


In [5]:
input_folder = 'Dataset/Full Mobility/Left hand'

#split with a ratio (train, val, test)
splitfolders.ratio(input_folder, output='Full Mobility/Left hand/',
                  seed=42, ratio=(.8, .15, .05),
                  group_prefix=None)

input_folder = 'Dataset/Full Mobility/Right hand'

#split with a ratio
splitfolders.ratio(input_folder, output='Full Mobility/Right hand/',
                  seed=42, ratio=(.8, .15, .05),
                  group_prefix=None)

input_folder = 'Dataset/Restricted mobility/Left hand'

#split with a ratio
splitfolders.ratio(input_folder, output='Restricted Mobility/Left hand/',
                  seed=42, ratio=(.8, .15, .05),
                  group_prefix=None)

input_folder = 'Dataset/Restricted mobility/Right hand'

#split with a ratio
splitfolders.ratio(input_folder, output='Restricted Mobility/Right hand/',
                  seed=42, ratio=(.8, .15, .05),
                  group_prefix=None)

Copying files: 492 files [00:05, 87.33 files/s]
Copying files: 502 files [00:04, 113.82 files/s]
Copying files: 484 files [00:05, 85.83 files/s]
Copying files: 495 files [00:05, 98.88 files/s]


In [6]:
def create_dataset(root_dir):
    """
    Creates an ImageFolder dataset from the specified root directory.
    """
    data_transforms = transforms.Compose([transforms.Resize((224, 224)),
                                          transforms.ToTensor(),
                                          transforms.RandomRotation(30),
                                          transforms.RandomHorizontalFlip(p=0.3),
                                          transforms.Normalize([0.485, 0.456, 0.406],
                                                               [0.229, 0.224, 0.225])])
    dataset = datasets.ImageFolder(root_dir, transform=data_transforms)
    return dataset

def combine_datasets(root_folders):
    """
    Combines datasets from multiple root folders into one dataset.
    """
    datasets = [create_dataset(folder) for folder in root_folders]
    combined_dataset = ConcatDataset(datasets)

    # Extract class-to-index mappings from individual datasets and merge them
    class_to_idx = {}
    for dataset in datasets:
        class_to_idx.update(dataset.class_to_idx)

    return combined_dataset, class_to_idx


def main():
  #choose which folders to include in test here
    root_folders = ['/content/Restricted Mobility/Left hand/train',
                    '/content/Restricted Mobility/Right hand/train',
                    '/content/Full Mobility/Left hand/train',
                    '/content/Full Mobility/Right hand/train']

    combined_dataset, combined_dataset.class_to_idx = combine_datasets(root_folders)

    print(f"Class to index mapping: {combined_dataset.class_to_idx}")
    return combined_dataset

def main_val():
  #choose which folders to include in test here
    root_folders = ['/content/Restricted Mobility/Left hand/val',
                    '/content/Restricted Mobility/Right hand/val',
                    '/content/Full Mobility/Left hand/val',
                    '/content/Full Mobility/Right hand/val']

    combined_dataset, combined_dataset.class_to_idx = combine_datasets(root_folders)

    print(f"Class to index mapping: {combined_dataset.class_to_idx}")
    return combined_dataset

def main_test():
  #choose which folders to include in test here
    root_folders = ['/content/Restricted Mobility/Left hand/test',
                    '/content/Restricted Mobility/Right hand/test',
                    '/content/Full Mobility/Left hand/test',
                    '/content/Full Mobility/Right hand/test']

    combined_dataset, combined_dataset.class_to_idx = combine_datasets(root_folders)

    print(f"Class to index mapping: {combined_dataset.class_to_idx}")
    return combined_dataset

combined_train_dataset = main()
combined_val_dataset = main_val()
combined_test_dataset = main_test()

Class to index mapping: {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'Space': 19, 'T': 20, 'U': 21, 'V': 22, 'W': 23, 'X': 24, 'Y': 25, 'Z': 26}
Class to index mapping: {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'Space': 19, 'T': 20, 'U': 21, 'V': 22, 'W': 23, 'X': 24, 'Y': 25, 'Z': 26}
Class to index mapping: {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'Space': 19, 'T': 20, 'U': 21, 'V': 22, 'W': 23, 'X': 24, 'Y': 25, 'Z': 26}


In [2]:
class SimplifiedModel(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(SimplifiedModel, self).__init__()

        # Load a pretrained DenseNet model
        densenet_model = models.densenet121(pretrained=True)

        # Use only the feature extractor part of DenseNet
        self.features = densenet_model.features

        # Modify the classifier part
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Dropout(p=0.3), #next test for this
            nn.Linear(1024, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# Create an instance of the SimplifiedModel
model = SimplifiedModel(in_channels=3,num_classes=27).to(device)
#print(model)
criterion = nn.CrossEntropyLoss()
opt = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

NameError: name 'nn' is not defined

In [36]:
#create loaders
train_loader = torch.utils.data.DataLoader(dataset=combined_train_dataset,
                                           batch_size=50,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=combined_test_dataset,
                                          batch_size=50,
                                          shuffle=False)

In [ ]:
from torch.optim.lr_scheduler import StepLR
from torch.utils.tensorboard import SummaryWriter

# Set up learning rate scheduler
scheduler = StepLR(opt, step_size=10, gamma=0.5)

# Training loop
def train(epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        logits = model(images)

        # Backpropagation and optimization
        loss = nn.functional.cross_entropy(logits, labels)
        loss.backward()

        # Apply gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        opt.step()
        opt.zero_grad()

        # Update running loss
        running_loss += loss.item()

        # Update accuracy
        predicted = torch.argmax(logits, -1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

        # Print live outputs
        print_every = 10
        if batch_idx % print_every == 0:
            print(f'Train Epoch {epoch+1}, Batch {batch_idx}/{len(train_loader)}, Loss: {running_loss/print_every:.3f}, Accuracy: {100 * correct / total:.2f}%')
            running_loss = 0.0
            correct = 0
            total = 0

# Testing loop
def test(epoch):
    model.eval()
    correct = 0
    total = 0
    test_loss = 0.0

    with torch.no_grad():
        for batch_idx, (images, labels) in enumerate(test_loader):
            images = images.to(device)
            labels = labels.to(device)

            logits = model(images)

            # Compute total correct so far
            predicted = torch.argmax(logits, -1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            # Update test loss
            loss = nn.functional.cross_entropy(logits, labels)
            test_loss += loss.item()

        # Print live outputs
        print(f'Test Epoch {epoch+1}, Loss: {test_loss/len(test_loader):.3f}, Accuracy: {100 * correct / total:.2f}%')

# Run training
epochs = 11
for epoch in range(epochs):
    train(epoch)
    test(epoch)
    scheduler.step()

Train Epoch 1, Batch 0/31, Loss: 0.344, Accuracy: 0.00%
Train Epoch 1, Batch 10/31, Loss: 2.882, Accuracy: 16.40%
Train Epoch 1, Batch 20/31, Loss: 2.211, Accuracy: 31.60%
Train Epoch 1, Batch 30/31, Loss: 1.764, Accuracy: 42.67%
Test Epoch 1, Loss: 2.151, Accuracy: 30.48%
Train Epoch 2, Batch 0/31, Loss: 0.139, Accuracy: 58.00%
Train Epoch 2, Batch 10/31, Loss: 1.317, Accuracy: 57.40%
Train Epoch 2, Batch 20/31, Loss: 1.295, Accuracy: 56.40%
Train Epoch 2, Batch 30/31, Loss: 1.195, Accuracy: 58.62%
Test Epoch 2, Loss: 1.750, Accuracy: 45.24%
Train Epoch 3, Batch 0/31, Loss: 0.103, Accuracy: 64.00%
Train Epoch 3, Batch 10/31, Loss: 1.040, Accuracy: 66.40%
Train Epoch 3, Batch 20/31, Loss: 0.958, Accuracy: 67.40%
Train Epoch 3, Batch 30/31, Loss: 0.865, Accuracy: 72.20%
Test Epoch 3, Loss: 0.927, Accuracy: 69.05%
Train Epoch 4, Batch 0/31, Loss: 0.078, Accuracy: 68.00%
Train Epoch 4, Batch 10/31, Loss: 0.735, Accuracy: 72.40%
Train Epoch 4, Batch 20/31, Loss: 0.769, Accuracy: 73.80%
Tra

In [29]:
torch.save(model.state_dict(), 'Densenet pretrained - test 4.pth')